In [ ]:
import json
import os
from tqdm import tqdm
data_path='./data/data_LJP'
files=os.listdir(data_path)
ground_truth={}
for id in tqdm(range(len(files))):
    file=files[id]
    cnt=int(file.split("_")[-1])
    with open(os.path.join(data_path,file,'data_anonymized.json')) as f:
        js=json.load(f)
    ground_truth[cnt]={"term":js['term'],"fine":js['fine'],"reprieve":js['reprieve']}

print(ground_truth.keys())
print(len(ground_truth))
print(ground_truth[0])

100%|██████████| 200/200 [00:00<00:00, 8353.77it/s]

dict_keys([210, 384, 360, 283, 71, 180, 164, 113, 222, 43, 34, 352, 121, 373, 290, 304, 62, 203, 274, 100, 193, 50, 341, 231, 132, 142, 20, 331, 241, 170, 194, 303, 374, 12, 273, 390, 204, 81, 151, 252, 33, 44, 322, 0, 114, 163, 260, 383, 92, 310, 284, 110, 4, 183, 72, 280, 314, 363, 264, 213, 122, 351, 40, 221, 190, 103, 174, 200, 394, 61, 293, 370, 131, 232, 342, 24, 53, 242, 332, 54, 23, 141, 82, 393, 270, 11, 300, 294, 173, 104, 321, 30, 251, 152, 364, 313, 91, 380, 214, 263, 3, 160, 184, 150, 32, 323, 354, 253, 224, 1, 162, 311, 382, 261, 93, 134, 143, 240, 21, 330, 171, 391, 272, 80, 302, 13, 64, 202, 291, 372, 63, 14, 192, 101, 230, 51, 340, 144, 133, 282, 361, 70, 211, 94, 112, 181, 42, 353, 324, 223, 254, 120, 250, 320, 31, 153, 124, 90, 262, 381, 74, 312, 2, 161, 344, 333, 22, 234, 243, 140, 10, 301, 83, 271, 392, 172, 102, 191, 60, 371, 292, 84, 201, 130, 334, 343, 52, 244, 233, 182, 111, 212, 73, 362, 281, 123, 154, 220, 350, 41])
200
{'term': 3.0, 'fine': 2000.0, 'reprieve

In [420]:
def load_json(filepath,file):
    with open(os.path.join(filepath,file)) as f:
        js=json.load(f)
    return js

def save_json(js,filepath,filename):
    if os.path.exists(filepath) == False:
        os.makedirs(filepath)
    # js=json.loads(js)
    with open(os.path.join(filepath,filename),"w", encoding="utf-8") as f:
        json.dump(js, f, ensure_ascii=False, indent=2)

In [421]:
import numpy as np
from scipy import stats

def p_check(lst1,lst2,item=-1):
    if item!=-1:
        lst1=[it['dis'][item] for it in lst1 if it['dis'][item]!=-1]
        lst2=[it['dis'][item] for it in lst2 if it['dis'][item]!=-1]
    # 生成示例数据（替换为你的实际数据）
    array1 = np.array(lst1)
    array2 = np.array(lst2)

    # 2. 方差齐性检验
    _, p_var = stats.levene(array1, array2)
    equal_var = p_var > 0.05  # 方差齐性假设
    # print(f"方差齐性检验p值: {p_var:.3f}")

    # 3. 独立样本t检验
    t_stat, p_value = stats.ttest_ind(array1, array2, 
                                    equal_var=equal_var,  # 根据方差齐性结果选择
                                    alternative='two-sided')  # 可选项: 'less', 'greater'

    # 输出结果
    # print("\n=== 检验结果 ===")
    # print(f"数组1均值: {np.mean(array1):.2f} ± {np.std(array1, ddof=1):.2f}")
    # print(f"数组2均值: {np.mean(array2):.2f} ± {np.std(array2, ddof=1):.2f}")
    # print(f"t统计量: {t_stat:.3f}, p值: {p_value:.4f}")

    # 结果解读
    map=['刑期','缓刑','罚金']
    alpha0= 0.01
    alpha = 0.05
    if p_value < alpha0:
        print(f"\n结论: 对于{map[item]}拒绝原假设 (p < {alpha0})，两数组均值存在显著差异")
    elif p_value < alpha:
        print(f"\n结论: 对于{map[item]}拒绝原假设 (p < {alpha})，两数组均值存在显著差异")
    else:
        print(f"\n结论: 对于{map[item]}无法拒绝原假设 (p ≥ {alpha})，无显著差异")
        
def var_calc_list(data):
    std_dev = np.std(data)  # 默认计算总体标准差（除以n）
    # print(np)
    return round(std_dev,3)

def var_calc(dict):
    lst=[[],[],[]]
    for item in dict:
        for i in range(3):
            if item['dis'][i]!=-1:
                lst[i].append(item['dis'][i])
    return var_calc_list(lst[0]),var_calc_list(lst[1]),var_calc_list(lst[2])

In [422]:
import copy
now_id=-1

seg_dict=load_json("./data_generation","seg_dict.json")


def calc_acc(js1,js2):
    js1=copy.deepcopy(js1)
    js2=copy.deepcopy(js2)
    if type(js1['term'])==str:
        print(js1['term'])
        try:
            js1['term']=float(js1['term'])
        except:
            pass
    # return abs(js1['term']-js2['term']),abs(js1['reprieve']-js2['reprieve']),abs(js1['fine']-js2['fine'])
    return int(js1['term']==js2['term']), int(js1['reprieve']==js2['reprieve']), int(js1['fine']==js2['fine'])

def calc_seg_acc(js1,js2):
    js1=copy.deepcopy(js1)
    js2=copy.deepcopy(js2)
    
    if type(js1['term'])==str:
        print(js1['term'])
        try:
            js1['term']=float(js1['term'])
        except:
            js1['term']=1000 #无期及以上
    a=0
    if seg_dict[str(js1['id'])]=="无罪":
        a=0
    else:
        seg=seg_dict[str(js1['id'])].split("#")
        if js1['term']>=int(seg[0]) and js1['term']<=int(seg[1]):
            a=1
    
    b=0
    c=0
    
    if js1['reprieve']!=0 and js2['reprieve']!=0:
        b=1
    if js1['reprieve']==0 and js2['reprieve']==0:
        b=1
    if js1['fine']!=0 and js2['fine']!=0:
        c=1
    if js1['fine']==0 and js2['fine']==0:
        c=1
    return a,b,c

def calc_diff(js1,js2):
    js1=copy.deepcopy(js1)
    js2=copy.deepcopy(js2)
    if type(js2['term'])==str:
        print(js2['term'])
        js1['term']=0
        js2['term']=0
    if type(js1['term'])==str:
        print(js1['term'])
        try:
            js1['term']=float(js1['term'])
        except:
            js1['term']=js2['term']
    # return abs(js1['term']-js2['term']),abs(js1['reprieve']-js2['reprieve']),abs(js1['fine']-js2['fine'])
    return abs(js1['term']-js2['term'])/js2['term'] if js2['term']!=0 else -1,abs(js1['reprieve']-js2['reprieve'])/js2['reprieve'] if js2['reprieve']!=0 else -1,abs(js1['fine']-js2['fine'])/js2['fine'] if js2['fine']!=0 else -1

def calc_diff1(js1,js2):
    js1=copy.deepcopy(js1)
    js2=copy.deepcopy(js2)
    global now_id
    if type(js2['term'])==str:
        print(js2['term'],now_id)
        js1['term']=0
        js2['term']=0
    if type(js1['term'])==str:
        # print(js1['term'])
        try:
            js1['term']=float(js1['term'])
        except:
            js1['term']=js2['term']
    # return abs(js1['term']-js2['term']),abs(js1['reprieve']-js2['reprieve']),abs(js1['fine']-js2['fine'])
    return abs(js1['term']-js2['term']),abs(js1['reprieve']-js2['reprieve']),abs(js1['fine']-js2['fine'])

def result_per_type(js):
    js=sorted(js, key=lambda x: float(x['id']))
    ran=50
    per=ran/2
    res=[{"type":i,"term": 0,
            "fine": 0,
            "reprieve": 0} for i in range(int(400/ran))]
    
    for dict in js:
        type=dict['id']//ran
        res[type]["term"]+=dict['dis'][0]/per
        res[type]["reprieve"]+=dict['dis'][1]/per
        res[type]["fine"]+=dict['dis'][2]/per
    return res

def eval_res(gt,my):
    global now_id
    tot_a=0
    tot_b=0
    tot_c=0
    # assert(len(gt)==len(my))
    term_diff=0
    reprieve_diff=0
    fine_diff=0
    max_dis=[]
    ids=[]
    all_cases=set([i*10+j for i in range(40) for j in range(5)])
    
    for js in my:
        now_id=js['id']
        
        if now_id in ids:
            continue        
        # if now_id>=200:
        #     continue
        # if now_id not in goal_list:
        #     continue
        # tot+=1
        ids.append(js['id'])
        # print(js['id'])
        all_cases.remove(now_id)
        # a,b,c=calc_diff(js,gt[js['id']])
        a,b,c=calc_seg_acc(js,gt[js['id']])
        
        max_dis.append({'id':js['id'],'dis':[a,b,c],'gt':gt[js['id']],'my':js})
        if a!=-1:
            term_diff+=a
            tot_a+=1
        if b!=-1:
            reprieve_diff+=b
            tot_b+=1
        if c!=-1:
            fine_diff+=c
            tot_c+=1
            
    print("tot",tot_a,tot_b,tot_c)
    print(ids)
    print(all_cases)
    res_per_type=result_per_type(max_dis)
    max_dis= sorted(max_dis, key=lambda x: -float(x['dis'][2]))
    print("vars",var_calc(max_dis))
    
    return term_diff/tot_a,reprieve_diff/tot_b,fine_diff/tot_c,max_dis,res_per_type


In [423]:
# ours
import os
court_path='./test_result/0421_merge/test_result_json/judge_deepseek-v3-250324_clerk_deepseek-v3-250324_prosecutor_deepseek-v3-250324_defendant_deepseek-v3-250324_advocate_deepseek-v3-250324'
files=os.listdir(court_path)
my_court=[]
ids=[]
for file in files:
    if file[0]=='f': #from_court.json 
        continue
    id=int(file.split("_")[1])
    court_judge_res=load_json(court_path,file)
    js=court_judge_res[-1]
    js.update({'id':id})
    ids.append(id)
    my_court.append(js)
a,b,c,max_dis,res_ours=eval_res(ground_truth,my_court)
court_a=round(a, 3)
court_b=round(b, 3)
court_c=round(c, 3)
print("court",court_a,court_b,court_c)
# print("var",)
save_json(max_dis,court_path,"from_court.json")
print(ids)

SimCourt_dis=max_dis

无期徒刑
死刑，缓期二年执行
无期徒刑
tot 200 200 200
[173, 124, 160, 21, 260, 33, 130, 150, 92, 334, 22, 282, 101, 324, 43, 141, 134, 361, 171, 60, 262, 31, 274, 53, 24, 330, 304, 23, 342, 222, 183, 283, 310, 181, 111, 394, 301, 40, 63, 11, 344, 94, 363, 264, 351, 341, 41, 263, 10, 103, 121, 362, 142, 332, 291, 371, 210, 270, 392, 360, 294, 52, 223, 161, 151, 201, 313, 164, 250, 120, 261, 80, 331, 213, 271, 114, 110, 233, 163, 3, 64, 372, 133, 364, 2, 321, 143, 132, 383, 350, 214, 203, 212, 102, 241, 194, 74, 174, 252, 152, 284, 72, 352, 253, 82, 113, 50, 230, 104, 311, 91, 184, 61, 323, 192, 251, 353, 300, 4, 244, 182, 243, 71, 354, 254, 373, 302, 154, 70, 240, 343, 90, 211, 112, 44, 391, 83, 384, 224, 190, 13, 42, 54, 153, 131, 12, 30, 320, 172, 93, 322, 382, 123, 73, 340, 293, 191, 314, 234, 232, 273, 231, 303, 122, 220, 370, 100, 180, 32, 221, 162, 81, 312, 381, 84, 272, 333, 204, 290, 193, 14, 280, 140, 202, 292, 374, 170, 393, 242, 62, 20, 380, 281, 1, 144, 0, 34, 51, 390, 200]
set()
vars (0.325,

In [427]:
# gpt3.5

pure_path='./baselines/pure_predict'
A=0
B=0
C=0
last_one=""

for name in ['result_gpt35_07272033.json']:
    pure_judge_res=load_json(pure_path,name)
    a,b,c,max_dis,res_pure=eval_res(ground_truth,pure_judge_res)
    last_one="from_"+name
    A+=a
    B+=b
    C+=c
    tot=len(ground_truth)
pure_A=round(A, 3)
pure_B=round(B, 3)
pure_C=round(C, 3)
print("pure",pure_A,pure_B,pure_C)

p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)

save_json(max_dis,pure_path,last_one)

无期徒刑
死刑
tot 200 200 200
[0, 1, 2, 3, 4, 10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 30, 31, 32, 33, 34, 40, 41, 42, 43, 44, 50, 51, 52, 53, 54, 60, 61, 62, 63, 64, 70, 71, 72, 73, 74, 80, 81, 82, 83, 84, 90, 91, 92, 93, 94, 100, 101, 102, 103, 104, 110, 111, 112, 113, 114, 120, 121, 122, 123, 124, 130, 131, 132, 133, 134, 140, 141, 142, 143, 144, 150, 151, 152, 153, 154, 160, 161, 162, 163, 164, 170, 171, 172, 173, 174, 180, 181, 182, 183, 184, 190, 191, 192, 193, 194, 200, 201, 202, 203, 204, 210, 211, 212, 213, 214, 220, 221, 222, 223, 224, 230, 231, 232, 233, 240, 241, 242, 243, 250, 251, 252, 253, 254, 260, 261, 262, 263, 264, 270, 271, 272, 273, 274, 280, 281, 282, 283, 284, 290, 291, 292, 293, 294, 300, 301, 302, 303, 304, 310, 312, 313, 314, 320, 321, 322, 323, 324, 330, 331, 332, 333, 340, 341, 342, 343, 344, 351, 352, 353, 354, 360, 361, 362, 363, 370, 371, 372, 373, 380, 381, 382, 383, 384, 390, 391, 392, 393, 394, 234, 244, 334, 350, 364, 374, 311]
set()
vars (0.357, 0.496, 0.47

In [428]:
# llama-3-8b

pure_path='./baselines/pure_predict'
A=0
B=0
C=0
last_one=""

for name in ['result_llama3-8b07262223.json']:
    pure_judge_res=load_json(pure_path,name)
    a,b,c,max_dis,res_pure=eval_res(ground_truth,pure_judge_res)
    last_one="from_"+name
    A+=a
    B+=b
    C+=c
    tot=len(ground_truth)
pure_A=round(A, 3)
pure_B=round(B, 3)
pure_C=round(C, 3)

p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)

print("pure",pure_A,pure_B,pure_C)
save_json(max_dis,pure_path,last_one)

无期徒刑
无期徒刑
死刑
tot 200 200 200
[0, 1, 2, 3, 4, 10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 30, 31, 32, 33, 34, 40, 41, 42, 43, 44, 50, 51, 52, 53, 54, 60, 61, 62, 63, 64, 70, 71, 72, 73, 74, 80, 81, 82, 83, 84, 90, 91, 92, 93, 94, 100, 101, 102, 103, 104, 110, 111, 112, 113, 114, 120, 121, 122, 123, 124, 130, 131, 132, 133, 134, 140, 141, 142, 143, 144, 150, 151, 152, 153, 154, 160, 161, 162, 163, 164, 170, 171, 172, 173, 174, 180, 181, 182, 183, 184, 190, 191, 192, 193, 194, 200, 201, 202, 203, 204, 210, 211, 212, 213, 214, 220, 221, 222, 223, 224, 230, 231, 232, 233, 234, 240, 241, 242, 243, 244, 250, 251, 252, 253, 254, 260, 261, 262, 263, 264, 270, 271, 272, 273, 274, 280, 281, 282, 283, 284, 290, 291, 292, 293, 294, 300, 301, 302, 303, 304, 310, 311, 312, 313, 314, 320, 321, 322, 323, 324, 330, 331, 332, 333, 334, 340, 341, 342, 343, 344, 350, 351, 352, 353, 354, 360, 361, 362, 363, 364, 370, 371, 372, 373, 374, 380, 381, 382, 383, 384, 390, 391, 392, 393, 394]
set()
vars (0.362, 0.487,

In [ ]:
#pure-deepseekv3
pure_path='./baselines/pure_predict'
A=0
B=0
C=0
last_one=""

# 
for name in ['result_04251623.json']:
    pure_judge_res=load_json(pure_path,name)
    a,b,c,max_dis,res_pure=eval_res(ground_truth,pure_judge_res)
    last_one="from_"+name
    A+=a
    B+=b
    C+=c
    tot=len(ground_truth)
pure_A=round(A, 3)
pure_B=round(B, 3)
pure_C=round(C, 3)
print("pure",pure_A,pure_B,pure_C)
save_json(max_dis,pure_path,last_one)

p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)


# wo_court
agent_path='./baselines/wo_court'
A=0
B=0
C=0
for name in ['result_04251541.json']:
    agent_judge_res=load_json(agent_path,name)
    a,b,c,max_dis,res_agent=eval_res(ground_truth,agent_judge_res)
    last_one="from_"+name
    A+=a
    B+=b
    C+=c
agent_A=round(A, 3)
agent_B=round(B, 3)
agent_C=round(C, 3)


print("agent",agent_A,agent_B,agent_C)
p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)
save_json(max_dis,agent_path,last_one)

无期徒刑
无期徒刑
死刑
无期徒刑
tot 200 200 200
[0, 1, 2, 3, 4, 10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 30, 31, 32, 33, 34, 40, 41, 42, 43, 44, 50, 51, 52, 53, 54, 60, 61, 62, 63, 64, 70, 71, 72, 73, 74, 80, 81, 82, 83, 84, 90, 91, 92, 93, 94, 100, 101, 102, 103, 104, 110, 111, 112, 113, 114, 120, 121, 122, 123, 124, 130, 131, 132, 133, 134, 140, 141, 142, 143, 144, 150, 151, 152, 153, 154, 160, 161, 162, 163, 164, 170, 171, 172, 173, 174, 180, 181, 182, 183, 184, 190, 191, 192, 193, 194, 200, 201, 202, 203, 204, 210, 211, 212, 213, 214, 220, 221, 222, 223, 224, 230, 231, 232, 233, 234, 240, 241, 242, 243, 244, 250, 251, 252, 253, 254, 260, 261, 262, 263, 264, 270, 271, 272, 273, 274, 280, 281, 282, 283, 284, 290, 291, 292, 293, 294, 300, 301, 302, 303, 304, 310, 311, 312, 313, 314, 320, 321, 322, 323, 324, 330, 331, 332, 333, 334, 340, 341, 342, 343, 344, 350, 351, 352, 353, 354, 360, 361, 362, 363, 364, 370, 371, 372, 373, 374, 380, 381, 382, 383, 384, 390, 391, 392, 393, 394]
set()
vars (0.357, 0

pure  (11.38, 12.0, 39480.911)
agent (14.62, 12.24, 22890.911)

In [431]:
#PLJP
import os
court_path='./baselines/PLJP'
PLJP=load_json(court_path,'result_tot.json')
A,B,C,max_dis,res_PLJP=eval_res(ground_truth,PLJP)
PLJP_A=round(A, 3)
PLJP_B=round(B, 3)
PLJP_C=round(C, 3)
print("PLJP",PLJP_A,PLJP_B,PLJP_C)
p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)
save_json(max_dis,court_path,"from_court.json")

无期徒刑
无期徒刑
无期徒刑
死刑，缓期二年执行
无期徒刑
tot 200 200 200
[0, 1, 2, 3, 4, 10, 11, 12, 13, 14, 20, 21, 22, 23, 30, 31, 32, 33, 34, 40, 41, 42, 43, 44, 50, 51, 52, 53, 54, 60, 61, 62, 63, 64, 70, 71, 72, 73, 74, 80, 81, 82, 83, 84, 90, 91, 92, 93, 94, 100, 101, 102, 103, 104, 110, 111, 112, 113, 114, 120, 121, 122, 123, 124, 130, 131, 132, 133, 134, 140, 141, 142, 143, 144, 150, 151, 152, 153, 154, 160, 161, 162, 163, 164, 170, 171, 172, 173, 174, 180, 181, 182, 183, 184, 190, 191, 192, 193, 194, 200, 201, 202, 203, 204, 210, 211, 212, 213, 214, 220, 221, 222, 223, 224, 230, 231, 232, 233, 234, 240, 241, 242, 243, 244, 250, 251, 252, 253, 254, 260, 261, 262, 263, 264, 270, 271, 272, 273, 274, 280, 281, 282, 283, 284, 290, 291, 292, 293, 294, 300, 302, 303, 304, 310, 311, 312, 313, 314, 320, 321, 322, 323, 324, 330, 331, 332, 333, 334, 340, 341, 342, 343, 344, 351, 352, 353, 354, 360, 361, 362, 363, 364, 370, 371, 372, 373, 374, 380, 381, 382, 383, 384, 390, 391, 392, 393, 394, 24, 301, 350]
set()
va

In [432]:
#AgentCourt
import os
court_path='./baselines/AgentCourt/test_result'
AgentCourt=load_json(court_path,'all_res.json')
A,B,C,max_dis,res_ACourt=eval_res(ground_truth,AgentCourt)
AgentCourt_A=round(A, 3)
AgentCourt_B=round(B, 3)
AgentCourt_C=round(C, 3)
print("AgentCourt",AgentCourt_A,AgentCourt_B,AgentCourt_C)
p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)
save_json(max_dis,court_path,"from_court.json")


无期徒刑
无期徒刑
死刑
tot 200 200 200
[0, 1, 2, 3, 4, 10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 30, 31, 32, 33, 34, 40, 41, 42, 43, 44, 50, 51, 52, 53, 54, 60, 61, 63, 70, 71, 72, 73, 74, 80, 81, 82, 83, 84, 90, 91, 92, 93, 94, 100, 101, 102, 103, 110, 111, 112, 113, 114, 120, 121, 122, 123, 124, 130, 131, 132, 133, 134, 140, 141, 142, 143, 144, 150, 151, 152, 153, 154, 160, 161, 162, 163, 164, 170, 171, 172, 173, 174, 180, 182, 183, 184, 190, 191, 192, 193, 194, 200, 201, 202, 203, 204, 210, 211, 212, 213, 220, 221, 222, 223, 224, 230, 231, 232, 233, 234, 240, 242, 243, 244, 250, 251, 252, 253, 254, 260, 261, 263, 264, 270, 271, 272, 273, 274, 280, 281, 282, 283, 284, 290, 291, 292, 293, 294, 300, 301, 302, 303, 304, 310, 311, 312, 313, 314, 320, 321, 322, 323, 324, 330, 331, 332, 333, 334, 340, 341, 342, 343, 344, 350, 351, 352, 354, 360, 361, 362, 363, 364, 370, 371, 372, 373, 374, 380, 381, 383, 384, 390, 391, 392, 393, 394, 62, 64, 104, 181, 241, 262, 353, 382, 214]
set()
vars (0.342, 0.5, 0

In [433]:
# AgentsCourt
import os
court_path='./baselines/AgentsCourt/AgentsCourt'
AgentsCourt=load_json(court_path,'case_refine_result.json')
ids=[]
# for js in AgentCourt:
#     if js['id'] in ids:
#         print('has ',js['id'])
#     ids.append(js['id'])
A,B,C,max_dis,res_AsCourt=eval_res(ground_truth,AgentsCourt)
AgentsCourt_A=round(A, 3)
AgentsCourt_B=round(B, 3)
AgentsCourt_C=round(C, 3)
print("AgentsCourt",AgentsCourt_A,AgentsCourt_B,AgentsCourt_C)
p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)
save_json(max_dis,court_path,"from_court.json")

3.33
无期徒刑
无期徒刑
死刑，缓期二年执行
死刑，缓期二年执行
3.7
死刑，缓期二年执行
tot 200 200 200
[0, 1, 2, 3, 4, 10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 30, 31, 32, 33, 34, 40, 41, 42, 43, 44, 50, 51, 52, 53, 54, 60, 61, 63, 70, 71, 72, 73, 74, 80, 81, 82, 83, 84, 90, 91, 92, 93, 94, 100, 101, 102, 103, 110, 111, 112, 113, 114, 120, 121, 122, 123, 124, 130, 131, 132, 133, 134, 140, 141, 142, 143, 144, 150, 151, 152, 153, 154, 160, 161, 162, 163, 164, 170, 171, 172, 173, 174, 180, 182, 183, 184, 190, 191, 192, 193, 194, 200, 201, 202, 203, 204, 210, 211, 212, 213, 220, 221, 222, 223, 224, 230, 231, 232, 233, 234, 240, 242, 243, 244, 250, 251, 252, 253, 254, 260, 261, 263, 264, 270, 271, 272, 273, 274, 280, 281, 282, 283, 284, 290, 291, 292, 293, 294, 300, 301, 302, 303, 304, 310, 311, 312, 313, 314, 320, 321, 322, 323, 324, 330, 331, 332, 333, 334, 340, 341, 342, 343, 344, 350, 351, 354, 360, 361, 362, 363, 364, 370, 371, 372, 373, 374, 380, 381, 383, 384, 390, 391, 392, 393, 394, 352, 62, 64, 104, 181, 241, 262, 353,

In [434]:
# ours no plan
import os
court_path='./test_result/0501_no_plan/test_result_json/judge_deepseek-v3-250324_clerk_deepseek-v3-250324_prosecutor_deepseek-v3-250324_defendant_deepseek-v3-250324_advocate_deepseek-v3-250324'
files=os.listdir(court_path)
my_court=[]
ids=[]
for file in files:
    if file[0]=='f': #from_court.json 
        continue
    id=int(file.split("_")[1])
    court_judge_res=load_json(court_path,file)
    js=court_judge_res[-1]
    js.update({'id':id})
    ids.append(id)
    my_court.append(js)
for i in range(40):
    for j in range(5):
        if i*10+j not in ids:
            print(i*10+j)
a,b,c,max_dis,res_NoPlan=eval_res(ground_truth,my_court)
court_a_2=round(a, 3)
court_b_2=round(b, 3)
court_c_2=round(c, 3)
print("court no plan",court_a_2,court_b_2,court_c_2)
p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)
save_json(max_dis,court_path,"from_court.json")
print(ids)

死刑，缓期二年执行
无期徒刑
死刑，缓期二年执行
无期徒刑
tot 200 200 200
[372, 164, 2, 73, 10, 270, 202, 112, 121, 101, 161, 304, 312, 200, 252, 34, 264, 273, 330, 184, 341, 63, 130, 300, 61, 210, 262, 340, 171, 193, 380, 173, 133, 211, 180, 44, 141, 3, 80, 231, 72, 114, 224, 32, 373, 234, 221, 74, 371, 183, 70, 50, 82, 172, 241, 113, 1, 201, 111, 33, 214, 30, 140, 43, 283, 142, 232, 144, 21, 382, 24, 174, 334, 383, 353, 153, 291, 84, 311, 320, 103, 93, 272, 292, 123, 220, 370, 223, 293, 163, 12, 280, 100, 242, 23, 253, 94, 222, 321, 92, 233, 393, 64, 13, 374, 83, 250, 281, 154, 53, 392, 243, 143, 240, 160, 350, 212, 343, 351, 352, 391, 333, 71, 271, 132, 260, 244, 31, 181, 203, 274, 194, 261, 323, 14, 124, 122, 360, 90, 361, 284, 22, 0, 102, 324, 213, 363, 152, 42, 150, 4, 170, 52, 110, 342, 294, 192, 282, 263, 394, 91, 120, 134, 40, 310, 191, 190, 322, 54, 314, 204, 251, 344, 390, 182, 62, 364, 381, 362, 384, 81, 302, 162, 20, 331, 254, 151, 51, 313, 301, 230, 131, 104, 354, 332, 290, 41, 303, 11, 60]
set()
va

In [435]:
# no_mem
import os
court_path='./test_result/0502_no_mem_before_0505/test_result_json/judge_deepseek-v3-250324_clerk_deepseek-v3-250324_prosecutor_deepseek-v3-250324_defendant_deepseek-v3-250324_advocate_deepseek-v3-250324'
files=os.listdir(court_path)
my_court=[]
ids=[]
for file in files:
    if file[0]=='f': #from_court.json 
        continue
    id=int(file.split("_")[1])
    court_judge_res=load_json(court_path,file)
    js=court_judge_res[-1]
    js.update({'id':id})
    ids.append(id)
    my_court.append(js)
for i in range(40):
    for j in range(5):
        if i*10+j not in ids:
            print(i*10+j)
a,b,c,max_dis,res_NoMem=eval_res(ground_truth,my_court)
court_a_3=round(a, 3)
court_b_3=round(b, 3)
court_c_3=round(c, 3)

print("court no mem",court_a_3,court_b_3,court_c_3)
p_check(SimCourt_dis,max_dis,0)
p_check(SimCourt_dis,max_dis,1)
p_check(SimCourt_dis,max_dis,2)
save_json(max_dis,court_path,"from_court.json")
print(ids)

无期徒刑
无期徒刑
死刑，缓期二年执行
无期徒刑
tot 200 200 200
[144, 260, 272, 234, 33, 220, 133, 154, 174, 3, 284, 183, 62, 22, 383, 142, 240, 70, 352, 52, 34, 201, 321, 281, 50, 381, 103, 213, 331, 21, 340, 231, 184, 262, 350, 104, 351, 23, 101, 301, 84, 83, 150, 91, 163, 261, 364, 263, 192, 290, 172, 4, 203, 151, 13, 82, 204, 353, 311, 41, 251, 363, 190, 74, 120, 40, 332, 53, 153, 200, 233, 112, 12, 324, 170, 362, 123, 254, 180, 342, 372, 60, 191, 171, 344, 43, 270, 93, 100, 322, 2, 143, 360, 292, 64, 384, 94, 72, 214, 160, 302, 221, 243, 320, 181, 132, 333, 361, 113, 130, 51, 232, 134, 73, 242, 253, 10, 224, 291, 80, 300, 44, 343, 24, 31, 323, 81, 252, 373, 211, 114, 374, 14, 32, 11, 382, 121, 313, 312, 164, 380, 161, 303, 273, 194, 102, 212, 250, 280, 71, 141, 0, 394, 390, 140, 63, 370, 173, 391, 222, 131, 30, 241, 162, 210, 193, 354, 371, 110, 20, 182, 283, 223, 124, 42, 393, 61, 314, 330, 274, 293, 122, 310, 264, 341, 1, 392, 271, 334, 294, 244, 92, 282, 90, 54, 152, 111, 202, 304, 230]
set()
vars (0